# **Estudo Link :**
 - https://medium.com/geleia/an%C3%A1lise-de-a%C3%A7%C3%B5es-com-python-7f6624939fb
 - https://opencodecom.net/post/2021-05-07-fronteira-eficiente/

In [15]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd

In [21]:
url = "https://yubb.com.br/investimentos/renda-fixa?collection_page=2&investment_type=cdb&months=12&principal=5000.0&sort_by=net_return"
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/ 97.0.4692.99 Safari/537.36"}
site1 = requests.get(url, headers=headers)
soup = bs(site1.content, 'html.parser')

In [32]:
def extrator(soup):
        bloco_informacoes = soup.find('div', {'class':'investmentCardContainer__body'})
        cards = bloco_informacoes.find_all('article', {'data-spec':'investments/card'})
        list_resultado_retorno = []
        list_resultado_rentabilidade = []
        list_resultado_tipo_rentabilidade = []
        list_resultado_empresa = []
        list_resultado_tag = []
        list_resultado_tipo = []
        list_resultado_rentabilidade_ano = []
        list_resultado_prazo = []
        list_resultado_distribuidor = []
        list_resultado_emissor = []

        for card in cards:
            info_head = card.find('header',{'class':'stack'})
            #retorno
            resultado_retorno = info_head.find('div', {'class':'results__netReturn'})
            resultado_retorno = resultado_retorno.find('span', {'class':'sugarish__whole'}).text
            resultado_retorno = resultado_retorno.replace(',', '.')
            list_resultado_retorno.append(resultado_retorno)
            #rentabilidade
            resultado_rentabilidade = info_head.find('div', {'class':'results__grossYield'})
            resultado_rentabilidade = resultado_rentabilidade.find('span',{'class':'sugarish__number'}).text
            resultado_rentabilidade = resultado_rentabilidade.replace(',', '.')
            list_resultado_rentabilidade.append(resultado_rentabilidade)
            #tipo de rentabilidade 
            resultado_tipo_rentabilidade = info_head.find('div', {'class':'results__grossYield'})
            resultado_tipo_rentabilidade = resultado_tipo_rentabilidade.find('span',{'class':'sugarish__cdi'}).text
            list_resultado_tipo_rentabilidade.append(resultado_tipo_rentabilidade)
            #empresa
            empresa = info_head.find('h3', {'class':'flex-stack font-size--s0'})
            empresa = empresa.text
            list_resultado_empresa.append(empresa)
            #tipo
            tipo = info_head.find('h4', {'class':'badge'}).text
            list_resultado_tipo.append(tipo)
            #tag
            try:
                tag_classificacao = info_head.find('span',{'class':'certification__tag'}).text
            except:
                tag_classificacao = info_head.find('span',{'class':'certification__tag'})
            list_resultado_tag.append(tag_classificacao)
            # --section
            info_section = card.find('section')
            table = info_section.find('table')
            tbody = table.find('tbody')
            #rentabilidade_ano
            rentabilidade_ano = tbody.find('th', text=re.compile('Rentabilidade líquida ao ano'))
            rentabilidade_ano = rentabilidade_ano.find_next('td').text
            list_resultado_rentabilidade_ano.append(rentabilidade_ano)
            #prazo_resgate
            prazo_resgate = tbody.find('th', text=re.compile('Prazo de resgate'))
            prazo_resgate = prazo_resgate.find_next('td').text
            list_resultado_prazo.append(prazo_resgate)
            #distribuidor
            distribuidor = tbody.find('th', text=re.compile('Distribuidor'))
            distribuidor = distribuidor.find_next('td').text
            list_resultado_distribuidor.append(distribuidor)
            #emissor
            emissor = tbody.find('th', text=re.compile('Emissor'))
            emissor = emissor.find_next('td').text
            list_resultado_emissor.append(emissor)

        return list_resultado_retorno, list_resultado_rentabilidade, list_resultado_tipo_rentabilidade, list_resultado_empresa, list_resultado_tag,\
               list_resultado_tipo, list_resultado_rentabilidade_ano, list_resultado_prazo, list_resultado_distribuidor,\
               list_resultado_emissor

In [33]:
list_resultado_retorno, list_resultado_rentabilidade, list_resultado_tipo_rentabilidade, list_resultado_empresa, list_resultado_tag, \
        list_resultado_tipo, list_resultado_rentabilidade_ano, list_resultado_prazo, list_resultado_distribuidor, \
        list_resultado_emissor = extrator(soup)

In [34]:
list_resultado_emissor

['Agibank',
 'Máxima',
 'Agibank',
 'Agibank',
 'Agibank',
 'Omni Banco',
 'Agibank',
 'BTG Pactual',
 'Banco Master',
 'PAN',
 'ABC',
 'BRK Financeira']

In [35]:
dicionario = {
    "retorno": list_resultado_retorno,
    "rentabilidade": list_resultado_rentabilidade,
    "tipo_rentabilidade": list_resultado_tipo_rentabilidade,
    "empresa": list_resultado_empresa,
    "tag": list_resultado_tag,
    "tipo": list_resultado_tipo,
    "rentabilidade_ano": list_resultado_rentabilidade_ano,
    "prazo": list_resultado_prazo,
    "distribuidor": list_resultado_distribuidor,
    "emissor": list_resultado_emissor
}

In [36]:
df = pd.DataFrame(dicionario) 

In [37]:
df

,retorno,rentabilidade,tipo_rentabilidade,empresa,tag,tipo,rentabilidade_ano,prazo,distribuidor,emissor
0,5.578,14.08,a.a.,"Agibank Prefixado 14,08% por Ágora Investimentos",FGC,CDB,"11,61% a.a.",17/10/2023,Ágora Investimentos,Agibank
1,5.578,14.07,a.a.,"Máxima IPCA + 8,01% por modalmais",FGC,CDB,"11,61% a.a.",17/10/2023,modalmais,Máxima
2,5.575,13.95,a.a.,"Agibank Prefixado 13,95% por Órama",FGC,CDB,"11,50% a.a.",18/10/2023,Órama,Agibank
3,5.573,13.95,a.a.,"Agibank Prefixado 13,95% por BTG Pactual",FGC,CDB,"11,50% a.a.",17/10/2023,BTG Pactual,Agibank
4,5.573,13.95,a.a.,"Agibank Prefixado 13,95% por Nova Futura Inves...",FGC,CDB,"11,50% a.a.",17/10/2023,Nova Futura Investimentos,Agibank
5,5.571,13.92,a.a.,"Omni Banco Prefixado 13,92% por BTG Pactual",FGC,CDB,"11,48% a.a.",17/10/2023,BTG Pactual,Omni Banco
6,5.571,13.85,a.a.,"Agibank Prefixado 13,85% por Guide Investimentos",FGC,CDB,"11,42% a.a.",18/10/2023,Guide Investimentos,Agibank
7,5.570,13.88,a.a.,"BTG Pactual IPCA + 7,83% por BTG Pactual",FGC,CDB,"11,45% a.a.",17/10/2023,BTG Pactual,BTG Pactual
8,5.569,13.86,a.a.,"Banco Master IPCA + 7,81% por Guide Investimentos",FGC,CDB,"11,43% a.a.",17/10/2023,Guide Investimentos,Banco Master
9,5.569,13.85,a.a.,"PAN IPCA + 7,8% por Nu invest",FGC,CDB,"11,42% a.a.",17/10/2023,Nu invest,PAN


In [47]:
df.max()

retorno                                       5.578
rentabilidade                                 14.08
tipo_rentabilidade                             a.a.
empresa               PAN IPCA + 7,8% por Nu invest
tag                                             FGC
tipo                                            CDB
rentabilidade_ano                       11,61% a.a.
prazo                                    18/10/2023
distribuidor                                  Órama
emissor                                         PAN
dtype: object

In [48]:
df[df['retorno']==df['retorno'].max()]

,retorno,rentabilidade,tipo_rentabilidade,empresa,tag,tipo,rentabilidade_ano,prazo,distribuidor,emissor
0,5.578,14.08,a.a.,"Agibank Prefixado 14,08% por Ágora Investimentos",FGC,CDB,"11,61% a.a.",17/10/2023,Ágora Investimentos,Agibank
1,5.578,14.07,a.a.,"Máxima IPCA + 8,01% por modalmais",FGC,CDB,"11,61% a.a.",17/10/2023,modalmais,Máxima
